In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import datetime
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append("..")

import slicer
from slicer import models

N_MINIBATCH = 64
N_EPOCHS = 20
LR = 0.01
PATIENCE = 3

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Read Data

In [ ]:
X_train = np.load("../data/X_train.npy").reshape((-1, 80, 322, 1))
Y_train = np.load("../data/Y_train.npy")
W_train = np.load("../data/W_train.npy")

X_test = np.load("../data/X_test.npy").reshape((-1, 80, 322, 1))
Y_test = np.load("../data/Y_test.npy")
W_test = np.load("../data/W_test.npy")

In [ ]:
max_print = 3
for idx in np.where(Y_train == 1)[0][1004:1004 + max_print]:
    plt.figure()
    plt.imshow(X_train[idx].reshape(80, 322), aspect="auto")

# Train

In [ ]:
model = models.CNN().model

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=False)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
history = model.fit(
    X_train,
    Y_train,
    batch_size=N_MINIBATCH,
    epochs=N_EPOCHS,
    validation_data=(X_test, Y_test),
    callbacks=[callback, tensorboard_callback],
)

# Inference

In [ ]:
import librosa
from scipy.ndimage import filters
from scipy import signal

from slicer import utils
from slicer.constants import SR
from slicer.constants import HOP_LENGTH
from slicer.constants import PEAK_PICKING_THRES
from slicer.constants import PINK_NOISE_DUR

from scipy.spatial import distance

class Slicer(object):
    def __init__(self):
        pass
    
    def do_the_slice(self):
        raise NotImplementedError
        
    def plot(self, nc, bounds, S, offset=0):
        # Plot novelty curve
        plt.figure(figsize=(20, 5))
        plt.plot(nc)
        for b in bounds:
            plt.vlines(b, ymin=-0.05, ymax=1.05, color='red')

        # Plot melspec with boundaries
        plt.figure(figsize=(20, 5))
        plt.imshow(S, aspect="auto")
        for b in bounds:
            plt.vlines(b + offset, ymin=0, ymax=80, color='red')
        
        

class Ulrich(Slicer):
    def __init__(self):
        super().__init__()
    
    def do_the_slice(self, mel, plot=False):
        # Preprocess
        mel = utils.to_db(utils.pad_pink_noise(mel))

        # Get patches and do inference
        X = utils.get_mel_patches(mel, 1, 322)
        Y = model.predict(X.reshape((-1, 80, 322, 1)))

        # Obtain boundaries
        bounds = utils.peak_picking(Y.flatten(), 0.68)

        # Plot results
        if plot:
            offset = int(PINK_NOISE_DUR * SR // HOP_LENGTH)
            self.plot(Y, bounds, mel, offset)

        return bounds
    

class Foote(Slicer):
    def __init__(self):
        super().__init__()
        
    def median_filter(self, X, M=8):
        """Median filter along the first axis of the feature matrix X."""
        for i in range(X.shape[1]):
            X[:, i] = filters.median_filter(X[:, i], size=M)
        return X
            
    def compute_ssm(self, X, metric="seuclidean"):
        """Computes the self-similarity matrix of X."""
        D = distance.pdist(X, metric=metric)
        D = distance.squareform(D)
        D /= D.max()
        return (1 - D) ** 2
    
    def compute_gaussian_krnl(self, M):
        """Creates a gaussian kernel following Foote's paper."""
        g = signal.gaussian(M, M // 3., sym=True)
        G = np.dot(g.reshape(-1, 1), g.reshape(1, -1))
        G[M // 2:, :M // 2] = -G[M // 2:, :M // 2]
        G[:M // 2, M // 2:] = -G[:M // 2, M // 2:]
        return G
    
    def compute_nc(self, X, G):
        """Computes the novelty curve from the self-similarity matrix X and
        the gaussian kernel G."""
        N = X.shape[0]
        M = G.shape[0]
        nc = np.zeros(N)

        for i in range(M // 2, N - M // 2 + 1):
            nc[i] = np.sum(X[i - M // 2:i + M // 2, i - M // 2:i + M // 2] * G)

        # Normalize
        nc += nc.min()
        nc /= nc.max()
        return nc
    
    def do_the_slice(self, mel, plot=False):
        """Main Foote function."""
        mel = mel.T
        meldb = utils.to_db(mel)
        meldb = self.median_filter(meldb, M=64)
        D = self.compute_ssm(meldb)
        G = self.compute_gaussian_krnl(256)
        nc = self.compute_nc(D, G)
        bounds = utils.peak_picking(nc.flatten(), 0.15)
        
        if plot:
            plt.figure()
            plt.imshow(D)
            plt.figure()
            plt.imshow(G)
            self.plot(nc, bounds, meldb.T)
        
        return bounds
            

AUDIO = "../trains.mp3"
# AUDIO = "../paranoid.mp3"
# AUDIO = "../bohemian.mp3"
bounds = Foote().do_the_slice(mel, plot=True)

In [ ]:
audio, _ = librosa.load(AUDIO, sr=slicer.constants.SR)
mel = utils.compute_melspec(audio)
bounds = Ulrich().do_the_slice(mel, plot=True)

In [ ]:
import IPython
c = librosa.clicks(frames=bounds, sr=SR, hop_length=HOP_LENGTH, length=len(audio))
IPython.display.Audio(audio + c, rate=SR)

In [ ]:
model.save('../models/cnn')

# Evaluate

In [ ]:
from slicer import evaluate

u_slicer = Foote()
df = evaluate.eval_slicer(u_slicer)

In [ ]:
df.mean()

In [ ]:
model = keras.models.load_model("../models/cnn")